# Chapter 12: Custom Models and Training with TensorFlow

Sejauh ini, kita telah menggunakan API tingkat tinggi TensorFlow, yaitu `tf.keras`. Meskipun sangat kuat, terkadang kita memerlukan kontrol lebih untuk menulis *loss function*, metrik, *layer*, atau bahkan *training loop* kustom.

Bab ini akan menyelami API tingkat rendah TensorFlow. Kita akan belajar cara:
* Menggunakan TensorFlow seperti NumPy untuk operasi tensor.
* Membuat komponen Keras kustom: *loss*, *layer*, metrik, dan model.
* Membuat *training loop* kustom dari awal.
* Memanfaatkan fitur pembuatan grafik otomatis TensorFlow dengan `tf.function` untuk meningkatkan kinerja.

## A Quick Tour of TensorFlow

Inti dari TensorFlow adalah pustaka komputasi numerik yang sangat mirip dengan NumPy, tetapi dengan dukungan GPU dan kemampuan komputasi terdistribusi. API-nya berpusat pada **tensor**, yang mengalir dari satu operasi ke operasi lainnya.

Kita dapat membuat dan memanipulasi tensor dengan mudah.

In [ ]:
import tensorflow as tf

# Membuat tensor
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

# Operasi dasar
print("Tensor + 10:\n", t + 10)
print("\nTensor kuadrat:\n", tf.square(t))
print("\nPerkalian matriks (t @ t_transpose):\n", t @ tf.transpose(t))

## Customizing Models and Training Algorithms

Sebagian besar komponen Keras dapat dikustomisasi, biasanya dengan menulis sebuah fungsi sederhana atau dengan membuat *subclass* dari kelas yang relevan.

### Custom Loss Functions
Jika Anda membutuhkan *loss function* yang tidak tersedia di Keras, Anda dapat membuatnya sendiri. Contoh: Huber Loss.

In [ ]:
# Fungsi untuk membuat Huber loss
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

# Penggunaannya saat kompilasi model
model.compile(loss=create_huber(2.0), optimizer="nadam")

### Custom Layers
Untuk lapisan tanpa bobot (*stateless*), kita bisa menggunakan `keras.layers.Lambda`. Untuk lapisan dengan bobot (*stateful*), kita perlu membuat *subclass* dari `keras.layers.Layer`.

In [ ]:
# Contoh custom layer stateful sederhana (Dense layer versi simpel)
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

### Custom Training Loops

Dalam kasus yang jarang terjadi di mana metode `fit()` tidak cukup fleksibel (misalnya, jika ingin menggunakan dua *optimizer* yang berbeda untuk bagian yang berbeda dari jaringan), kita dapat menulis *training loop* kita sendiri. Ini memberi kita kontrol penuh atas proses pelatihan.

In [ ]:
# Contoh konseptual training loop kustom
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error

for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}/{n_epochs}")
    for step in range(1, n_steps + 1):
        # Ambil batch data (contoh acak)
        indices = np.random.randint(len(X_train), size=batch_size)
        X_batch, y_batch = X_train[indices], y_train[indices]

        # Hitung gradien di dalam GradientTape
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)

        # Terapkan gradien
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

## TensorFlow Functions and Graphs

Untuk meningkatkan kinerja, TensorFlow dapat mengubah fungsi Python menjadi grafik komputasi statis. Ini dilakukan dengan menggunakan `tf.function` sebagai dekorator atau fungsi.

Ketika sebuah fungsi didekorasi dengan `@tf.function`, TensorFlow akan melakukan *tracing* saat pertama kali dipanggil. Selama *tracing*, TensorFlow menganalisis kode Python dan membangun grafik komputasi yang setara dan dioptimalkan. Panggilan berikutnya akan menggunakan grafik ini, yang jauh lebih cepat daripada menjalankan kode Python baris per baris.

In [ ]:
@tf.function
def tf_cube(x):
    return x ** 3

print(tf_cube(tf.constant(2.0)))
print(tf_cube(2))